In [17]:
import re,os
import json
import ast,nltk
import datetime
from stemming.porter2 import stem
from collections import defaultdict
from string import punctuation
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import logging
import numpy as np
import time
from gensim.models import Word2Vec,KeyedVectors
from sklearn.preprocessing import Imputer

In [2]:
training_data = []
# Articles that has category given
with open('articles.json','r') as data_file:
    line = data_file.readline()
    while line:
        line = ast.literal_eval(line)
        training_data.append(line)
        line = data_file.readline()
    data_file.close()

In [3]:
testing_data = []
# Articles that has no category given
with open('articles_wo.json','r') as data_file:
    line = data_file.readline()
    while line:
        line = ast.literal_eval(line)
        testing_data.append(line)
        line = data_file.readline()
    data_file.close()

In [4]:
print len(training_data)

10369


In [5]:
# function to parse sentences to words and remove stopwords from sentence
def sentence_to_wordlist( sentence, remove_stopwords=True ):
    text = re.sub("[^a-zA-Z]"," ",sentence)
    words = text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

In [6]:
# function to parse text to sentences using tokenizer mentioned above
def text_to_sentences(text,tokenizer,remove_stopwords=True):
    raw_sentences = tokenizer.tokenize(text.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append( sentence_to_wordlist( raw_sentence, \
              remove_stopwords ))
    return sentences

In [7]:
words = []
classes = []
documents = []
no_classes = []
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# loop through each sentence in our training data
for pattern in training_data:
    # tokenize each word in the sentence
    words.extend(nltk.word_tokenize(pattern['desc']))
    documents += [[stem(str(word)) for word in sentence] \
                        for sentence in [text_to_sentences(pattern['desc'], tokenizer)]]
    classes.append(pattern['category'])
    if pattern['category'] not in no_classes:
        no_classes.append(pattern['category'])

In [8]:
test_documents = []
for pattern in testing_data:
    documents += [[stem(str(word)) for word in sentence] \
                          for sentence in [text_to_sentences(pattern['desc'],tokenizer)]]

In [9]:
output = []
# create an empty array for our output
output_empty = [0] * len(no_classes)
for Class in classes:
    output_row = list(output_empty)
    output_row[no_classes.index(Class)] = 1
    output.append(output_row)

In [10]:
stops = set(stopwords.words("english"))
words = [stem(w.lower()) for w in words if w not in stops]
words = list(set(words))

In [59]:
print len(words)

80053


In [11]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 1  # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 4           # Context window size                                                                                    
downsampling = 1e-5   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)

print "Training model..."
model = Word2Vec(documents, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=False)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "100features_1minwords_4context"
model.save(model_name)
#model.save_word2vec_format(model_name,binary=False)

2017-10-31 15:50:27,238 : INFO : collecting all words and their counts
2017-10-31 15:50:27,239 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-31 15:50:27,414 : INFO : PROGRESS: at sentence #10000, processed 204500 words, keeping 182852 word types
2017-10-31 15:50:27,427 : INFO : collected 190351 word types from a corpus of 213395 raw words and 10852 sentences
2017-10-31 15:50:27,428 : INFO : Loading a fresh vocabulary


Training model...


2017-10-31 15:50:28,432 : INFO : min_count=1 retains 190351 unique words (100% of original 190351, drops 0)
2017-10-31 15:50:28,433 : INFO : min_count=1 leaves 213395 word corpus (100% of original 213395, drops 0)
2017-10-31 15:50:28,990 : INFO : deleting the raw counts dictionary of 190351 items
2017-10-31 15:50:28,998 : INFO : sample=0.001 downsamples 1 most-common words
2017-10-31 15:50:28,999 : INFO : downsampling leaves estimated 212687 word corpus (99.7% of prior 213395)
2017-10-31 15:50:29,001 : INFO : estimated required memory for 190351 words and 100 dimensions: 247456300 bytes
2017-10-31 15:50:29,646 : INFO : resetting layer weights
2017-10-31 15:50:31,864 : INFO : training model with 4 workers on 190351 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4
2017-10-31 15:50:31,865 : INFO : expecting 10852 sentences, matching count from corpus used for vocabulary survey
2017-10-31 15:50:32,883 : INFO : PROGRESS: at 63.39% examples, 670627 words/s, in_qs

In [12]:
def makeFeatureVec(words,model,num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float128")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #sorted_set = set(model.sort_vocab())
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [13]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float128")
    # 
    # Loop through the reviews
    for review in reviews:
    # Print a status message every 1000th review
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(reviews))

    # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
    # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [14]:
DataVecs = getAvgFeatureVecs( documents, model, num_features )

Review 0 of 10852
Review 1000 of 10852
Review 2000 of 10852
Review 3000 of 10852
Review 4000 of 10852
Review 5000 of 10852
Review 6000 of 10852
Review 7000 of 10852
Review 8000 of 10852
Review 9000 of 10852
Review 10000 of 10852


In [15]:
print DataVecs.shape

(10852, 100)


In [18]:
X_train = Imputer().fit_transform(DataVecs[:len(training_data)])
X_test = Imputer().fit_transform(DataVecs[len(training_data):])

In [19]:

# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)
 
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

def think(x, show_details=False):
    #x = bow(sentence.lower(), words, show_details)
    #if show_details:
    #    print ("sentence:", sentence, "\n bow:", x)
    # input layer is our bag of words
    l0 = x
    # matrix multiplication of input and hidden layer
    l1 = sigmoid(np.dot(l0, synapse_0))
    # output layer
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2

In [20]:
# ANN and Gradient Descent code from https://iamtrask.github.io//2015/07/27/python-network-part2/
def train(X, y, hidden_neurons=10, alpha=1, epochs=50000, dropout=False, dropout_percent=0.5):

    print ("Training with %s neurons, alpha:%s, dropout:%s %s" % (hidden_neurons, str(alpha), dropout, dropout_percent if dropout else '') )
    print ("Input matrix: %sx%s    Output matrix: %sx%s" % (len(X),len(X[0]),1, len(no_classes)) )
    np.random.seed(1)

    last_mean_error = 1
    # randomly initialize our weights with mean 0
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
    synapse_1 = 2*np.random.random((hidden_neurons, len(no_classes))) - 1

    prev_synapse_0_weight_update = np.zeros_like(synapse_0)
    prev_synapse_1_weight_update = np.zeros_like(synapse_1)

    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)
        
    for j in iter(range(epochs+1)):

        # Feed forward through layers 0, 1, and 2
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))
                
        if(dropout):
            layer_1 *= np.random.binomial([np.ones((len(X),hidden_neurons))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))

        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # how much did we miss the target value?
        layer_2_error = y - layer_2

        if (j% 10000) == 0 and j > 5000:
            # if this 10k iteration's error is greater than the last iteration, break out
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error))) )
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print ("break:", np.mean(np.abs(layer_2_error)), ">", last_mean_error )
                break
                
        # in what direction is the target value?
        # were we really sure? if so, don't change too much.
        layer_2_delta = layer_2_error * sigmoid_output_to_derivative(layer_2)

        # how much did each l1 value contribute to the l2 error (according to the weights)?
        layer_1_error = layer_2_delta.dot(synapse_1.T)

        # in what direction is the target l1?
        # were we really sure? if so, don't change too much.
        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)
        
        synapse_1_weight_update = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight_update = (layer_0.T.dot(layer_1_delta))
        
        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight_update > 0)+0) - ((prev_synapse_0_weight_update > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight_update > 0)+0) - ((prev_synapse_1_weight_update > 0) + 0))        
        
        synapse_1 += alpha * synapse_1_weight_update
        synapse_0 += alpha * synapse_0_weight_update
        
        prev_synapse_0_weight_update = synapse_0_weight_update
        prev_synapse_1_weight_update = synapse_1_weight_update

    now = datetime.datetime.now()

    # persist synapses
    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': no_classes
              }
    synapse_file = "synapses.json"

    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", synapse_file)

In [21]:
#X = np.array(DataVecs)
y = np.array(output)

start_time = time.time()

train(X_train, y, hidden_neurons=10, alpha=0.1, epochs=10000, dropout=False, dropout_percent=0.1)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

Training with 10 neurons, alpha:0.1, dropout:False 
Input matrix: 10369x100    Output matrix: 1x20
delta after 10000 iterations:0.0692545086315
('saved synapses to:', 'synapses.json')
('processing time:', 326.55551195144653, 'seconds')


In [22]:
# probability threshold
ERROR_THRESHOLD = 0.01
# load our calculated synapse values
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])



In [26]:
def classify(vec, show_details=False):
    results = think(vec, show_details)
    #print results
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[no_classes[r[0]],r[1]] for r in results]
    #print ("%s \n classification: %s" % (sentence, return_results))
    return return_results

In [24]:
print classes[0]

politics


In [25]:
for vec in X_test:
    res = classify(X_test[0])
    print res

[  1.00000000e+000   7.36052383e-128   1.52489876e-046   8.28115100e-151
   3.83233949e-144   7.19932234e-120   3.50048849e-127   4.14029635e-140
   4.49421595e-163   2.10519511e-137   2.54433141e-062   3.73151491e-155
   1.13974292e-137   7.86277584e-052   1.38256933e-165   1.51050887e-097
   6.47299382e-056   1.78673886e-155   4.25125898e-164   5.96559822e-126]
[['politics', 1.0]]
